In [56]:
import json
from config import settings
import os


import polars as pl
from polars import col as c
from polars import selectors as cs
import networkx as nx
from networkx_function import generate_nx_edge
from typing_extensions import Union
import numpy as np
import scipy as sp

# from distflow_algorithm import DistFlow
import time
import pandapower as pp
import pandapower.networks as pn
from polars_function import (
    get_transfo_admittance,
    get_transfo_impedance,
    get_transfo_conductance,
    get_transfo_imaginary_component,
)

from data_connector import pandapower_to_distflow
from general_function import duckdb_to_dict, dict_to_duckdb, pl_to_dict
from networkx_function import get_all_edge_data, generate_shortest_path_length_matrix
import graphblas as gb
from utility.parser_utility import duckdb_to_changes_schema

from distflow_schema import DistFlowSchema

# Useless outside jupiternotebook because in settings.py a line that changes the directory to src for ipynb
os.chdir(os.getcwd().replace("/src", ""))

## Import Schema from duckdb

In [57]:
file_names: dict[str, str] = json.load(open(settings.INPUT_FILE_NAMES))
changes_schema = duckdb_to_changes_schema(file_path=file_names["duckdb_output"])
list(changes_schema.__dict__.keys())

Read and validate tables from matlab_grid.db file: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.22it/s]


['heartbeat',
 'resource',
 'equipment',
 'terminal',
 'busbar_section',
 'branch',
 'branch_parameter_event',
 'geo_event',
 'switch',
 'switch_event',
 'transformer',
 'transformer_end',
 'transformer_parameter_event',
 'tap',
 'tap_event',
 'bess',
 'energy_consumer',
 'external_network',
 'generating_unit',
 'container',
 'client',
 'substation',
 'base_voltage',
 'connectivity_node',
 'connectivity',
 'measurement',
 'measurement_point',
 'measurement_span']

In [165]:
resource_data = changes_schema.resource.filter(
    (c("concrete_class") == "branch")
    | (c("concrete_class") == "switch")
    | (c("concrete_class") == "tranfsormer")
    | (c("concrete_class") == "external_network")
).select(
    c("uuid"), c("dso_code").alias("element_id"), c("concrete_class").alias("type")
)

## Add row index to connectivity_node to give the number of node
## Join connectivity_node with connectivity to get the eq_fk
## Create the connectivity_node dictionnary with side+eq_fk as key and node index as value
connectivity_node: dict[float, str] = pl_to_dict(
    changes_schema.connectivity_node.with_row_index()
    .join(changes_schema.connectivity, left_on="uuid", right_on="cn_fk", how="left")
    .select((c("side") + c("eq_fk")).alias("eq_fk_side"), c("index"))
)

## Add node from and node to for each edge with side+eq_fk
resource_data = resource_data.with_columns(
    ("t1" + c("uuid"))
    .replace_strict(connectivity_node, default=None)
    .alias(
        "u_of_edge"
    ),  # Replace side+eq_fk with node number from connectivity for equipment
    ("t2" + c("uuid"))
    .replace_strict(connectivity_node, default=None)
    .alias("v_of_edge"),
)
# Add branch parameter
branch = changes_schema.branch_parameter_event[["uuid", "eq_fk", "r", "x", "b", "g"]]

# Add branch parameter to line_data
resource_data = resource_data.join(
    branch, left_on="uuid", right_on="eq_fk", how="left"
).drop("uuid_right")
## Add n_tranfo to 1, useless, because automatically added by add_table from class DistFlowSchema and by default value is 1
# resource_data = resource_data.with_columns(pl.lit(1).alias("n_tranfo"))

In [161]:
## Transfo in pu
s_base = 1e6  # 1 MVA for distribution

In [ ]:
test = DistFlowSchema()
test.add_table(edge_data=resource_data)

In [86]:
# Generate the node dict with uuid for each node
connectivity_node: dict[float, str] = pl_to_dict(
    changes_schema.connectivity_node.with_row_index().select(
        c("uuid").alias("cn_fk"), c("index").alias("node_number")
    )
)

## Generate the node number from the connectivity table with the cn_fk uuid
test = line_data.with_columns(
    c("cn_fk").replace_strict(connectivity_node, default=None).alias("node_number")
)

ColumnNotFoundError: cn_fk

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'with_columns' <---
DF ["uuid", "element_id", "type", "u_of_edge"]; PROJECT */9 COLUMNS

In [103]:
# Generate the node dict with uuid for each node
connectivity_node: dict[float, str] = pl_to_dict(
    changes_schema.connectivity_node.with_row_index().select(c("uuid"), c("index"))
)

## Generate the node number from the connectivity table with the cn_fk uuid
line_data = changes_schema.connectivity.with_columns(
    c("cn_fk").replace_strict(connectivity_node, default=None).alias("node_number")
)

# Slack node uuid = df941fce-ceda-5874-ab63-5c8af9bec38b, came from cn_fk of external_network

## Get the slack node number with the id of external network
slack_node_id = line_data.filter(c("eq_class") == "external_network")["node_number"][0]

In [104]:
line_data

diff,uuid,start_heartbeat,end_heartbeat,start,end,eq_fk,side,eq_class,abstraction_fk,cn_fk,container_fk,indirect,node_number
str,str,"datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]",str,str,str,str,str,str,bool,i64
"""+""","""a517aafd-14bc-441a-b243-cf63c5…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""b4fdf239-cfe2-5453-91c3-84c1b2…","""t1""","""branch""","""physical""","""ba84d70a-80d7-590e-b112-f9c4b5…",null,false,1
"""+""","""d370b75d-1d91-4005-8807-3d9b8c…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""3edce6fc-e164-5b3e-9b0d-8a1c86…","""t1""","""branch""","""physical""","""078656ed-79f8-53a1-a67a-bb8f53…",null,false,2
"""+""","""d190a4fd-4295-4a7c-8c92-796ae1…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""c173d7f3-f7ca-5f44-98d8-63c1aa…","""t1""","""branch""","""physical""","""af72457f-f983-5eeb-a635-0609f4…",null,false,4
"""+""","""021b361f-aacb-4dbb-a977-f4d2b0…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""c33afba9-0379-5d6a-9ca1-743cf7…","""t1""","""branch""","""physical""","""dbd2411e-1e87-5956-86d9-d69ee7…",null,false,5
"""+""","""5c4b2cd4-e815-4a2f-801a-d5d1dd…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""020bd95c-9320-5560-9627-4bbe19…","""t1""","""branch""","""physical""","""2db84a3d-aa74-5b4f-866a-331318…",null,false,6
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""+""","""74c69851-f366-4707-acee-fe1f74…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""c8a03e72-a228-5007-ad70-3cc3ab…","""t1""","""switch""","""physical""","""c2247320-9fc2-538a-ba64-3ac70e…",null,false,3
"""+""","""6347fbc0-3423-4f16-a5bd-f7d349…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""4f44a759-906d-568d-95b7-a57f77…","""t1""","""switch""","""physical""","""8a7f105e-71f3-5101-8b4f-1a9007…",null,false,9
"""+""","""2439e929-8699-4f5d-b7ca-98ab2f…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""c8a03e72-a228-5007-ad70-3cc3ab…","""t2""","""switch""","""physical""","""df941fce-ceda-5874-ab63-5c8af9…",null,false,0


In [59]:
## u, v and unpivot
# line_data.pivot(index="eq_class", on="side", values="node_number")
## add r,x,b,g in pu

## import type (switch, branch, transformer) from table

## add n_trafo
# distflow_data = line_data.filter().pivot.....

In [60]:
# edge_data: pl.DataFrame = line_data
# u_of_edge: int = pt.Field(dtype=pl.Int32)
# v_of_edge: int = pt.Field(dtype=pl.Int32)
# r_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
# x_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
# b_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
# g_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
# n_transfo: float = pt.Field(dtype=pl.Float64, default=1.0)
# type: TYPES = pt.Field(dtype=pl.Utf8, constraints=literal_constraint(pt.field, TYPES))

## Filter data and create distflowschema

TYPES = Literal["branch", "transformer", "switch"]
class EdgeData(pt.Model):
    u_of_edge: int = pt.Field(dtype=pl.Int32)
    v_of_edge: int = pt.Field(dtype=pl.Int32)
    r_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
    x_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
    b_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
    g_pu: float = pt.Field(dtype=pl.Float64, default=0.0)
    n_transfo: float = pt.Field(dtype=pl.Float64, default=1.0)
    type: TYPES = pt.Field(dtype=pl.Utf8, constraints=literal_constraint(pt.field, TYPES))

edge_data

In [61]:
# edge_data = 1
# test = DistFlowSchema(edge_data)

In [69]:
# edge_data = test.edge_data
changes_schema.resource

diff,uuid,start_heartbeat,end_heartbeat,start,end,dso_code,concrete_class,name,feeder_fk,metadata,owner
str,str,"datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]",str,str,str,str,str,str
"""+""","""b4fdf239-cfe2-5453-91c3-84c1b2…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_1""","""branch""",null,null,null,null
"""+""","""3edce6fc-e164-5b3e-9b0d-8a1c86…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_2""","""branch""",null,null,null,null
"""+""","""c173d7f3-f7ca-5f44-98d8-63c1aa…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_4""","""branch""",null,null,null,null
"""+""","""c33afba9-0379-5d6a-9ca1-743cf7…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_5""","""branch""",null,null,null,null
"""+""","""020bd95c-9320-5560-9627-4bbe19…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_6""","""branch""",null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…
"""+""","""b2c29241-7bfa-4515-aebc-07b62b…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""node_number_54""","""energy_consumer""",null,null,null,null
"""+""","""4e3a5eb1-0a0b-40f7-9995-043be5…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""node_number_57""","""energy_consumer""",null,null,null,null
"""+""","""c8a03e72-a228-5007-ad70-3cc3ab…",2025-02-04 10:32:48.945352 UTC,null,1960-01-01 00:00:00 UTC,null,"""line_3""","""switch""",null,null,null,null


switch

branch

branch_parameter_event

In [63]:
line_data = changes_schema.resource.filter(
    (c("concrete_class") == "branch") | (c("concrete_class") == "switch")
).select(c("uuid"), c("dso_code").alias("line_id"), c("concrete_class").alias("type"))
print(line_data)
cn_mapping = pl_to_dict(
    changes_schema.connectivity.select(
        pl.concat_str("eq_fk", "side").alias("id"), "uuid"
    )
)
## Add uuid of t1 and t2 for resource
line_data = line_data.with_columns(
    (c("uuid") + "t1")
    .replace_strict(cn_mapping, default=None)
    .alias("u_of_edge"),  # Replace eq_fk with uuid from connectivity for branch
    (c("uuid") + "t2").replace_strict(cn_mapping, default=None).alias("v_of_edge"),
)

# Add branch parameter
branch = changes_schema.branch_parameter_event[["uuid", "eq_fk", "r", "x", "b", "g"]]
# changes_schema.branch_parameter_event.select(
#     c("uuid"),
#     # c("source_fk"),
#     c("eq_fk"),
#     c("r"),
#     c("r0"),
#     c("x"),
#     c("x0"),
#     c("b"),
#     c("b0"),
#     c("g"),
#     c("g0"),
# )
# Add branch parameter to line_data
line_data = line_data.join(branch, left_on="uuid", right_on="eq_fk", how="left").drop(
    "uuid_right"
)

shape: (57, 3)
┌─────────────────────────────────┬─────────┬────────┐
│ uuid                            ┆ line_id ┆ type   │
│ ---                             ┆ ---     ┆ ---    │
│ str                             ┆ str     ┆ str    │
╞═════════════════════════════════╪═════════╪════════╡
│ b4fdf239-cfe2-5453-91c3-84c1b2… ┆ line_1  ┆ branch │
│ 3edce6fc-e164-5b3e-9b0d-8a1c86… ┆ line_2  ┆ branch │
│ c173d7f3-f7ca-5f44-98d8-63c1aa… ┆ line_4  ┆ branch │
│ c33afba9-0379-5d6a-9ca1-743cf7… ┆ line_5  ┆ branch │
│ 020bd95c-9320-5560-9627-4bbe19… ┆ line_6  ┆ branch │
│ …                               ┆ …       ┆ …      │
│ 34edd04e-a827-5cf8-b233-43d285… ┆ line_55 ┆ branch │
│ 9b34c19a-8b3e-54c6-9cd9-834bb5… ┆ line_56 ┆ branch │
│ ec28307e-8860-52db-9611-713cad… ┆ line_57 ┆ branch │
│ c8a03e72-a228-5007-ad70-3cc3ab… ┆ line_3  ┆ switch │
│ 4f44a759-906d-568d-95b7-a57f77… ┆ line_9  ┆ switch │
└─────────────────────────────────┴─────────┴────────┘


In [64]:
line_data

uuid,line_id,type,u_of_edge,v_of_edge,r,x,b,g
str,str,str,str,str,f64,f64,f64,f64
"""b4fdf239-cfe2-5453-91c3-84c1b2…","""line_1""","""branch""","""a517aafd-14bc-441a-b243-cf63c5…","""97f67c24-31fc-4ddf-8ca2-4fd768…",0.0031,0.0018075,0.000005,0.0
"""3edce6fc-e164-5b3e-9b0d-8a1c86…","""line_2""","""branch""","""d370b75d-1d91-4005-8807-3d9b8c…","""3e174ad0-b046-4c3c-9b13-5b0b53…",0.03275,0.01834,0.000056,0.0
"""c173d7f3-f7ca-5f44-98d8-63c1aa…","""line_4""","""branch""","""d190a4fd-4295-4a7c-8c92-796ae1…","""ef08765c-ddb8-420d-b9ab-4bc2f0…",0.04848,0.0014576,0.000001,0.0
"""c33afba9-0379-5d6a-9ca1-743cf7…","""line_5""","""branch""","""021b361f-aacb-4dbb-a977-f4d2b0…","""4404297a-9802-4ce1-8a0c-b3f843…",0.00124,0.000723,0.000002,0.0
"""020bd95c-9320-5560-9627-4bbe19…","""line_6""","""branch""","""5c4b2cd4-e815-4a2f-801a-d5d1dd…","""cbc3f456-193d-49cc-9e0e-7a96c0…",0.006625,0.00371,0.000011,0.0
…,…,…,…,…,…,…,…,…
"""34edd04e-a827-5cf8-b233-43d285…","""line_55""","""branch""","""c2bc64bd-9c9d-4f06-93ab-dc8b36…","""ba270f09-f0ee-4da9-91b2-9442c1…",0.000868,0.0005061,0.000001,0.0
"""9b34c19a-8b3e-54c6-9cd9-834bb5…","""line_56""","""branch""","""2ef67bc3-dedf-4908-9f93-002143…","""f7f9f5b2-ae53-4786-9935-f4ee1a…",0.05151,0.0015487,0.000002,0.0
"""ec28307e-8860-52db-9611-713cad…","""line_57""","""branch""","""7559c42b-ebc7-46c4-9734-e9b668…","""95680be7-4036-4a7e-91b6-b701e4…",0.03939,0.0011843,0.000001,0.0


In [65]:
node_id_list = (
    line_data.unpivot(on=["u_of_edge", "v_of_edge"], value_name="node_id")
    .unique("node_id", keep="first")["node_id"]
    .to_list()
)
node_id_to_nb_mapping = dict(zip(node_id_list, range(len(node_id_list))))
node_nb_to_id_mapping = dict(zip(range(len(node_id_list)), node_id_list))

line_data = line_data.with_columns(
    c(col).replace_strict(node_id_to_nb_mapping, default=None).alias(col)
    for col in ["u_of_edge", "v_of_edge"]
)
# ext_grid_nb = node_id_to_nb_mapping[ext_grid_id]

In [66]:
node_id_list

['f7f9f5b2-ae53-4786-9935-f4ee1a319d4c',
 '8d4f9a03-b38b-45d7-98f4-71df57e7f1b2',
 '254af29e-99ea-41a7-b06b-1d513f059b05',
 '218077d8-b950-4cd1-ae25-aad62758c649',
 '486ca599-f0a7-4167-a89f-b9a167407352',
 'b869fbeb-5d96-472f-9e7c-d027c3aada8c',
 '4a4bb787-2786-449e-90fe-07299012251c',
 '5c4b2cd4-e815-4a2f-801a-d5d1dd11fc47',
 '6ad6c6f7-aa52-4662-92e2-61551cdcbb6b',
 'cbc3f456-193d-49cc-9e0e-7a96c0f6439e',
 'ea19d1d1-bb31-420d-b811-1042a9be14d5',
 '5c3e42fe-7623-4065-ac0d-6df4f91ab12b',
 'd190a4fd-4295-4a7c-8c92-796ae117695b',
 '397c151c-9c43-407c-8176-e5584d58ad6e',
 '170b4b2d-320f-49f4-b551-c98e59a82953',
 'bad8c34f-c16f-4a00-ab64-b56c082aab79',
 'fa7bd651-a797-460a-abcb-c948b2c615a2',
 '45e1755a-9c9e-4464-aa6b-ecaf3d2f6095',
 'c2bc64bd-9c9d-4f06-93ab-dc8b36832a9f',
 '24d6a18d-a0f2-443b-8b4b-ea2b951ab1a3',
 '2439e929-8699-4f5d-b7ca-98ab2fd95911',
 'b451f77a-8f7c-4c40-8677-d46b9a93b8bf',
 '0397b202-3754-45bc-8424-ebc53ec561ae',
 '7f33fb8f-3c4f-44a5-b1b5-249c86cee991',
 '3e174ad0-b046-

In [67]:
# # from state_estimation_algorithm import StateEstimationData


# class StateEstimationData:
#     def __init__(self, changes_schema: dict):
#         self.line_data: pl.DataFrame
#         self.node_data: pl.DataFrame
#         self.changes_schema = changes_schema

#         self.generate_line_data()
#     ### TODO when code is ok
#     def generate_line_data(self):
#         self.line_data = self.changes_schema.resource.filter(
#             (c("concrete_class") == "branch") | (c("concrete_class") == "switch")
#         ).select(
#             c("uuid"), c("dso_code").alias("line_id"), c("concrete_class").alias("type")
#         )
# stateestimationdata = StateEstimationData(changes_schema=changes_schema)

In [68]:
ext_grid_id = "1"
n = 0.95
v_ext_grid_sq = 1.05

line_data: pl.DataFrame = pl.DataFrame(
    {
        "line_id": np.arange(1, 13).astype(str),
        "type": ["branch"] * 10 + ["transformer"] * 2,
        "u_of_edge": ["2", "2", "2", "4", "5", "6", "7", "10", "12", "13", "8", "9"],
        "v_of_edge": ["11", "1", "3", "1", "4", "4", "8", "9", "11", "11", "4", "8"],
        "n_transfo": [1.0] * 10 + [n**2, n**2],
        "x_pu": np.arange(1, 13) * 2e-3,
        "r_pu": np.arange(1, 13) * 1e-3,
        "b_pu": list(np.arange(1, 11) * -1e-3) + [0.001, 0.001],
        "g_pu": [0.0] * 10 + [0.001, 0.001],
    }
)

node_data: pl.DataFrame = pl.DataFrame(
    {
        "node_id": np.arange(1, 14).astype(str),
        "v_base": [400.0] * 7 + [200.0, 400.0, 400.0, 200.0, 100.0, 100.0],
        "p_node_pu": np.array([0, 10, 2, 0, 0, -2, 0, 7, 25, 0, 100, 0.5, 10]) * 1e-2,
        "q_node_pu": np.array([0, 0, 0.2, 2, 0, -0.2, 0, 0.7, 0, 0, 0, 0.5, 1]) * 1e-2,
    },
    strict=False,
)

np.set_printoptions(linewidth=200)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq

s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

NameError: name 'DistFlow' is not defined

In [ ]:
s_base = 1e6
net: pp.pandapowerNet = pp.from_pickle("data/input_grid/modified_cigre_network_lv.p")
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)

f: 0.012090921401977539


In [ ]:
ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0] ** 2
node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow: DistFlow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

In [ ]:
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq
s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

numpy: 5.9482645988464355
graphblas: 0.09949016571044922


In [ ]:
line_res_pp = pl.DataFrame(
    {
        "node_id": list(net.line["to_bus"].astype(str)),
        "p_pp": list(net.res_line["p_from_mw"]),
        "q_pp": list(net.res_line["q_from_mvar"]),
    }
)

node_res_pp = pl.from_pandas(net.res_bus["vm_pu"], include_index=True).select(
    c("index").cast(pl.Utf8).alias("node_id"), "vm_pu"
)

result = (
    pl.DataFrame(
        {
            "v": np.sqrt(v),
            "p": np.real(s_flow),
            "q": np.imag(s_flow),
        }
    )
    .with_row_index(name="node_id")
    .with_columns(
        c("node_id")
        .replace_strict(distflow.node_nb_to_id_mapping, default=None)
        .alias("node_id")
    )
    .join(line_res_pp, on="node_id", how="left")
    .join(node_res_pp, on="node_id", how="left")
    .with_columns(
        ((c("v") - c("vm_pu")) / c("vm_pu")).abs().alias("diff_v"),
        ((c("p") - c("p_pp")) / c("p_pp")).abs().alias("diff_p"),
        ((c("q") - c("q_pp")) / c("p_pp")).abs().alias("diff_q"),
    )
)
print(result["diff_v", "diff_p", "diff_q"].max())

shape: (1, 3)
┌──────────┬──────────┬──────────┐
│ diff_v   ┆ diff_p   ┆ diff_q   │
│ ---      ┆ ---      ┆ ---      │
│ f64      ┆ f64      ┆ f64      │
╞══════════╪══════════╪══════════╡
│ 0.000392 ┆ 0.000182 ┆ 0.010306 │
└──────────┴──────────┴──────────┘


In [ ]:
net: pp.pandapowerNet = pp.from_pickle("data/input_grid/synthesized_grid.p")
net.trafo["i0_percent"] = 2
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)

f: 0.5935556888580322


In [ ]:
s_base = 1e6
ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0] ** 2

node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

In [ ]:
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq
s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

numpy: 9.821000337600708
graphblas: 0.08703160285949707


In [ ]:
line_data = line_data.group_by("u_of_edge", "v_of_edge").agg(
    c("name", "n_transfo", "type").first(),
    c("r_pu", "x_pu").first() / c("r_pu", "x_pu").count(),
    c("g_pu", "b_pu").first() * c("g_pu", "b_pu").count(),
)